# Task 3

This notebook is part of Milestone 3, Question 3. You can work on this notebook on your laptop to develop your machine learning model using all the learnings from the previous courses. At the end of this notebook, when you are ready to train the model, you will need to obtain the hyperparameters from the hyperparameter tuning job that you will run in Milestone 3 Question 4 (i.e., the notebook named `Milestone3-Task4.ipynb`).

PS: To speed up the process, you can test the model without the hyperparameters first. Once other team members obtain the hyperparameters, you can retrain the model using those hyperparameters and test it again.

In [3]:
# I asked them to use their laptop so they already got all these packages from previous courses.
# %pip install joblib scikit-learn matplotlib s3fs

# Imports

In [1]:
import numpy as np
import pandas as pd
from joblib import dump, load
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
plt.style.use('ggplot')
plt.rcParams.update({'font.size': 16, 'axes.labelweight': 'bold', 'figure.figsize': (8,6)})
## add any other additional packages that you need. You are free to use any packages for vizualization.

## Part 1:

Recall as a final goal of this project. We want to build and deploy ensemble machine learning models in the cloud, where features are outputs of different climate models and the target is the actual rainfall observation. In this milestone, you'll actually build these ensemble machine learning models in the cloud.  

**Your tasks:**

1. Read the data CSV from your s3 bucket. 
2. Drop rows with nans. 
3. Split the data into train (80%) and test (20%) portions with `random_state=123`. 
4. Carry out EDA of your choice on the train split. 
5. Train ensemble machine learning model using `RandomForestRegressor` and evaluate with metric of your choice (e.g., `RMSE`) by considering `Observed` as the target column. 
6. Discuss your results. Are you getting better results with ensemble models compared to the individual climate models? 

> Recall that individual columns in the data are predictions of different climate models. 

## Step 1 : Read the data CSV from your s3 bucket.

In [2]:
## Remember by default it looks for credentials in home directory. 
## Make sure your updated credentials are in home directory
## or pass credentials explicitly and pass as storage_options=aws_credentials (not a good idea)
# aws_credentials = {"key": "","secret": "","token":""}
# replace with s3 path to your data
df = pd.read_csv("s3://mds-s3-3-arjun/output/ml_data_SYD.csv", index_col=0, parse_dates=True)

In [3]:
## Use your ML skills to get from step 1 to step 6
df

,ACCESS-CM2,ACCESS-ESM1-5,AWI-ESM-1-1-LR,BCC-CSM2-MR,BCC-ESM1,CMCC-CM2-HR4,CMCC-CM2-SR5,CMCC-ESM2,CanESM5,EC-Earth3-Veg-LR,...,MPI-ESM-1-2-HAM,MPI-ESM1-2-HR,MPI-ESM1-2-LR,MRI-ESM2-0,NESM3,NorESM2-LM,NorESM2-MM,SAM0-UNICON,TaiESM1,observed rainfall
time,,,,,,,,,,,,,,,,,,,,,
1889-01-01,0.040427,1.814552,3.557933e+01,4.268112e+00,1.107466e-03,1.141054e+01,3.322009e-08,2.668800,1.321215,1.515293,...,4.244226e-13,1.390174e-13,6.537884e-05,3.445495e-06,1.576096e+01,4.759651e-05,2.451075,0.221324,2.257933,0.006612
1889-01-02,0.073777,0.303965,4.596520e+00,1.190141e+00,1.015323e-04,4.014984e+00,1.312700e+00,0.946211,2.788724,4.771376,...,4.409552e+00,1.222283e-01,1.049131e-13,4.791993e-09,3.675510e-01,4.350863e-01,0.477231,3.757179,2.287381,0.090422
1889-01-03,0.232656,0.019976,5.927467e+00,1.003845e-09,1.760345e-05,9.660564e+00,9.103720e+00,0.431999,0.003672,4.233980,...,2.269300e-01,3.762301e-01,9.758706e-14,6.912302e-01,1.562869e-01,9.561101e+00,0.023083,0.253357,1.199909,1.401452
1889-01-04,0.911319,13.623777,8.029624e+00,8.225226e-02,1.808932e-01,3.951528e+00,1.317160e+01,0.368693,0.013578,15.252495,...,2.344586e-02,4.214019e-01,7.060915e-03,3.835721e-02,2.472225e-07,5.301038e-01,0.002699,2.185453,2.106737,14.869798
1889-01-05,0.698013,0.021048,2.132686e+00,2.496841e+00,4.708019e-09,2.766362e+00,1.822940e+01,0.339267,0.002468,11.920357,...,4.270161e-13,1.879692e-01,4.504985e+00,3.506923e-07,1.949792e-13,1.460928e-10,0.001026,2.766507,1.763335,0.467628
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014-12-27,0.033748,0.123476,1.451179e+00,3.852845e+01,2.061717e-03,8.179260e-09,1.171263e-02,0.090786,59.895054,5.071783,...,4.726998e-13,1.326889e-01,1.827857e+00,6.912631e-03,2.171327e-03,1.620489e+00,2.084252,0.868046,17.444923,0.037472
2014-12-28,0.094198,2.645497,4.249335e+01,5.833800e-01,5.939502e-09,8.146937e-01,4.938900e-01,0.000000,0.512632,1.578188,...,4.609420e-13,1.644482e+00,7.242920e-01,2.836752e-03,1.344768e+01,2.391160e+00,1.644527,0.782258,1.569647,0.158061
2014-12-29,0.005964,3.041667,2.898325e+00,9.359547e-02,2.000051e-08,2.532205e-01,1.306046e+00,0.000002,37.169670,1.565885,...,2.016156e+01,1.506439e+00,1.049481e-01,8.137182e+00,2.547820e+01,1.987695e-12,0.205036,2.140724,1.444630,0.025719


## Step 2: Drop rows with nans.

In [4]:
data = df.dropna()

In [6]:
data.shape

(45989, 26)

## Step 3: Split the data into train (80%) and test (20%).

In [5]:
train_df, test_df = train_test_split(data, test_size=0.2, random_state=123)

## Step 4: Carry out EDA of your choice on the train split.

In [13]:
# Todo

## Step 5: rain ensemble machine learning model using RandomForestRegressor

In [6]:
# Separate features and target
X_train = train_df.drop('observed rainfall', axis=1)
y_train = train_df['observed rainfall']
X_test = test_df.drop('observed rainfall', axis=1)
y_test = test_df['observed rainfall']

In [7]:
rf = RandomForestRegressor(random_state=123)
rf.fit(X_train, y_train)

RandomForestRegressor(random_state=123)

In [8]:
# Make prediction on train
y_pred_train = rf.predict(X_train)

# Make predictions on test
y_pred_test = rf.predict(X_test)

In [9]:
rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
print(f"RMSE for test data: {rmse_train}")

rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
print(f"RMSE for test data: {rmse_test}")

RMSE for test data: 3.2415357124566713
RMSE for test data: 8.081429130013323


## Step 6 - Discuss your results.

In [10]:
# Compare the results of the ensemble model with individual climate models:

rmse_df = pd.DataFrame({
    "Model": X_test.columns,
    "Train RMSE" : [mean_squared_error(y_train, X_train[column], squared=False) for column in X_train.columns],
    "Test RMSE": [mean_squared_error(y_test, X_test[column], squared=False) for column in X_test.columns]
}).set_index("Model").sort_values("Test RMSE")
%timeit rmse_df

26.8 ns ± 0.304 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [11]:
rmse_df

,Train RMSE,Test RMSE
Model,,
FGOALS-g3,9.497266,8.825898
KIOST-ESM,9.364012,8.929471
MPI-ESM1-2-HR,9.697459,9.136171
NESM3,9.559615,9.238419
MPI-ESM1-2-LR,9.907773,9.358796
GFDL-CM4,10.151643,9.460040
MRI-ESM2-0,9.716749,9.494019
EC-Earth3-Veg-LR,10.125208,9.560820
NorESM2-LM,10.072092,9.801393


Comparing the RF ensemble model's performance to the individual climate models, we can see that the RF model outperforms all individual models in terms of RMSE for both the training and test datasets. The RF model's Test RMSE of 8.081429130013323 is lower than the best performing individual model (FGOALS-g3) with a Test RMSE of 8.825898.

In conclusion, we are getting better results with the ensemble model (Random Forest) compared to the individual climate models, as evidenced by the lower RMSE values.

## Part 2:

### Preparation for deploying model next week

***NOTE: Complete Question 4 (`Milestone3-task4.ipynb`) from the milestone 3 before coming here***

We’ve found the best hyperparameter settings with MLlib (from the Question 4 from milestone3), here we then use the same hyperparameters to train a scikit-learn model. 

In [12]:
# Just replace ___ with the numbers you found from Milestone3-task4.ipynb
model = RandomForestRegressor(n_estimators=100, max_depth=5, random_state = 123)
model.fit(X_train, y_train)

RandomForestRegressor(max_depth=5)

In [13]:
print(f"Train RMSE: {mean_squared_error(y_train, model.predict(X_train), squared=False):.2f}")
print(f" Test RMSE: {mean_squared_error(y_test, model.predict(X_test), squared=False):.2f}")

Train RMSE: 8.11
 Test RMSE: 7.81


In [14]:
# ready to deploy
# where this model is saved? Understand the concept of relative path.
dump(model, "model.joblib")

['model.joblib']

***Upload model.joblib to s3 under output folder. You choose how you want to upload it (using CLI, SDK, or web console).*** Web console is also completely fine as it is a small file.